In [ ]:

# Импорты необходимых библиотек
import random
from kaggle_environments import make, evaluate
import numpy as np
import pandas as pd


In [3]:

# Определение агентов с различными стратегиями

# Агент, всегда играющий "камень"
def rock_agent(observation, configuration):
    return 0  # Всегда возвращает "камень"

# Агент с рандомной стратегией
def random_agent(observation, configuration):
    return random.randint(0, 2)  # Случайный выбор

# Агент, повторяющий последний ход противника
def mimic_agent(observation, configuration):
    if observation.step == 0:
        return 0  # По умолчанию "камень"
    return observation.lastOpponentAction  # Копирует действие оппонента

# Агент, обыгрывающий последнее действие противника
def beat_last_action_agent(observation, configuration):
    if observation.step == 0:
        return 1  # По умолчанию "бумага"
    return (observation.lastOpponentAction + 1) % 3  # Бьёт прошлый ход

# Агент, проигрывающий последнее действие противника
def lose_last_action_agent(observation, configuration):
    if observation.step == 0:
        return 2  # По умолчанию "ножницы"
    return (observation.lastOpponentAction + 2) % 3  # Проигрывает последнему ходу

# Агент с цикличной стратегией
def cyclical_agent(observation, configuration):
    return observation.step % 3  # Последовательно играет "камень", "бумага", "ножницы"

# Агент с вероятностным выбором (с весами)
def biased_random_agent(observation, configuration):
    return random.choices([0, 1, 2], weights=[0.5, 0.3, 0.2])[0]  # Увеличенная вероятность выбора "камня"

# Агент, реагирующий на ход противника
def reactive_agent(observation, configuration):
    if observation.step == 0:
        return 0  # По умолчанию "камень"
    if observation.lastOpponentAction == 0:
        return 1  # Реакция "бумага" на "камень"
    elif observation.lastOpponentAction == 1:
        return 2  # Реакция "ножницы" на "бумагу"
    else:
        return 0  # Реакция "камень" на "ножницы"

# Агент, всегда играющий "бумагу"
def always_paper_agent(observation, configuration):
    return 1  # Всегда "бумага"

# Агент, всегда играющий "ножницы"
def always_scissors_agent(observation, configuration):
    return 2  # Всегда "ножницы"

# Агент, играющий наперед против цикличного
def anti_cyclical_agent(observation, configuration):
    return (observation.step + 1) % 3  # На шаг впереди цикличного агента

# Агент, выбирающий действия для ничьей
def defensive_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)  # Случайный первый ход
    return observation.lastOpponentAction  # Повторяет для ничьей

# Агрессивный агент, бьющий самый популярный ход
def aggressive_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    counts = [observation.action_histogram.count(i) for i in range(3)]
    most_common = counts.index(max(counts))
    return (most_common + 1) % 3  # Бьёт самый частый ход

# Агент, обучающийся на истории противника
def adaptive_agent(observation, configuration):
    if observation.step < 2:
        return random.randint(0, 2)
    return (observation.lastOpponentAction + random.choice([-1, 0, 1])) % 3  # Прогнозирует следующий ход


In [4]:

# Функция для проведения турнира между агентами
def run_tournament(agents, rounds=10):
    # Создаем окружение для игры "камень-ножницы-бумага"
    env = make("rps", debug=True)
    scores = {agent.__name__: 0 for agent in agents}  # Инициализируем счет для каждого агента
    
    # Сравниваем всех агентов друг с другом
    for i, agent1 in enumerate(agents):
        for j, agent2 in enumerate(agents):
            if i >= j:  # Избегаем дублирования матчей
                continue
            outcomes = evaluate(
                "rps",
                [agent1, agent2],
                num_episodes=rounds
            )
            # Обновляем счет только если результаты корректны
            for outcome in outcomes:
                if outcome[0] is not None and outcome[1] is not None:
                    scores[agent1.__name__] += 1 if outcome[0] > outcome[1] else 0
                    scores[agent2.__name__] += 1 if outcome[1] > outcome[0] else 0
    
    return scores


In [6]:

# Список всех агентов для участия в турнире
agents = [
    rock_agent,
    random_agent,
    mimic_agent,
    beat_last_action_agent,
    lose_last_action_agent,
    cyclical_agent,
    biased_random_agent,
    reactive_agent,
    always_paper_agent,
    always_scissors_agent,
    anti_cyclical_agent,
    defensive_agent,
    aggressive_agent,
    adaptive_agent
]

# Проведение турнира и вывод результатов
scores = run_tournament(agents)
scores_df = pd.DataFrame(scores.items(), columns=["Агент", "Счет"]).sort_values(by="Счет", ascending=False)
scores_df


,Агент,Счет
7,reactive_agent,73
3,beat_last_action_agent,61
6,biased_random_agent,41
4,lose_last_action_agent,40
10,anti_cyclical_agent,38
8,always_paper_agent,33
1,random_agent,29
0,rock_agent,25
5,cyclical_agent,25
9,always_scissors_agent,24
